In [2]:
import string
import nltk
import enchant
from spello.model import SpellCorrectionModel
import textstat

nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\lbiondo\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\lbiondo\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [3]:
#load spell checker model
sp = SpellCorrectionModel(language='en')
sp.load('./spello_model/en_large.pkl')

c:\Users\lbiondo\Anaconda3\envs\tesi\lib\site-packages\spello\model.py:301: UserWarning: This model was saved on spell<1.3.0. As such due to a bug in previous versions, none of customisations made to the config at the time of training were saved along with the model. It is recommended to load the model, apply all required customizations to config and save it again. E.g.

from spello.model import SpellCorrectionModel 
sp = SpellCorrectionModel(language='en')  
sp.load('/home/ubuntu/model.pkl')
sp.config.min_length_for_spellcorrection = 4 # default is 3
sp.config.max_length_for_spellcorrection = 12 # default is 15
sp.save(model_save_dir='/home/ubuntu/')

After this the model will load without any warnings

  warnings.warn("This model was saved on spell<1.3.0. As such due to a bug in previous versions, "


In [4]:
def removePunctuationFromTokenized(contentsTokenized):
    excludePuncuation = set(string.punctuation)
    
    # manually add additional punctuation to remove
    doubleSingleQuote = '\'\''
    doubleDash = '--'
    doubleTick = '``'

    excludePuncuation.add(doubleSingleQuote)
    excludePuncuation.add(doubleDash)
    excludePuncuation.add(doubleTick)

    filteredContents = [word for word in contentsTokenized if word not in excludePuncuation]
    return filteredContents

#### Lexical Diversity

In [5]:
def lexical_diversity(text):
   return (len(set(text)) / len(text))

#### Percentage of Uppercased and Lowercased Words 

In [6]:
def percentage_uppercased(text_tokenized):
    islower = 0;
    isupper = 0;
    total_words = len(text_tokenized);

    for word in text_tokenized:      
        if word.islower():
            islower +=1         
        elif word.isupper():
            isupper+=1         

    return (islower/total_words,isupper/total_words)

#### Spell Checking Against Dictionary

In [7]:
def spell_checker_1(text_tokenized):
    d = enchant.Dict("en_US")
    mistakes = 0
    for word in text_tokenized:
        if not d.check(word):
            mistakes+=1
    return mistakes        

#### Spell Checking with ML model

In [8]:
def spell_checker_2(raw_text,text_tokenized):
    corrected = sp.spell_correct(raw_text)
    i = 0
    for w in text_tokenized:
        if(w in corrected['correction_dict']):
            i+=1
    return i

#### Unrecognized Words by POS Tagger

In [9]:
def unrecognized_by_pos(text_tokenized):
    text_tagged = nltk.pos_tag(text_tokenized,tagset='universal')
    unknown = 0
    for t in text_tagged:
         if t[1] == "X":
                #return t
                unknown += 1
    return unknown/len(text_tagged)            

#### Average Sentence Length

In [10]:
import doctest
import re


def avg_sentence_length(text):
    
    terminating_punct = "[!?.]"
    punct = r"\W"  # non-word characters
    sentences = [
        s.strip()  # without trailing whitespace
        for s in re.split(
            terminating_punct,
            "".join(text).replace("\n", " "),  # text as 1 string
        )
        if s.strip()  # non-empty
    ]

    def wordcount(s):
        """Split sentence s on punctuation
        and return number of non-empty words
        """
        return len([w for w in re.split(punct, s) if w])
    #map each sentece to its wordcount then sum all the wordcounts
    return sum(map(wordcount, sentences)) / len(sentences)


In [11]:
path = "./data/input/prose.txt"

with open(path, "r") as ifile:
    raw_text = ifile.read()
    
#raw_text = "Sample text to try functions"    
text_tokenized = removePunctuationFromTokenized(nltk.word_tokenize(raw_text))
lower,upper = percentage_uppercased(text_tokenized)
print("Lexical diversity: " + "{0:.3f}".format(lexical_diversity(text_tokenized)))
print("Lowercase sentences: "+ "{0:.3f}".format(lower))
print("Uppercase sentences: "+ "{0:.3f}".format(upper))
print("Spelling mistakes 1: "  + str(spell_checker_1(text_tokenized)))
print("Spelling mistakes 2: "  + str(spell_checker_2(raw_text,text_tokenized)))
print("Unrecognized by POS tagger: "+str(unrecognized_by_pos(text_tokenized)))
print("Average sentence length: " +"{0:.3f}".format(avg_sentence_length(raw_text)))
print("Readability (CLI): " + str(textstat.coleman_liau_index(raw_text)))
print("Readability (ARI): " + str(textstat.automated_readability_index(raw_text)))

Lexical diversity: 0.426
Lowercase sentences: 0.925
Uppercase sentences: 0.001
Spelling mistakes 1: 23
Spelling mistakes 2: 32
Unrecognized by POS tagger: 0.0
Average sentence length: 16.172
Readability (CLI): 8.13
Readability (ARI): 11.8
